**TENSORFLOW INSTALLATION**

In [26]:
!pip install tensorflow
!pip install --root-user-action=ignore
!pip install --upgrade tensorflow
!pip install tensorflow-gpu
!pip install --upgrade setuptools wheel
!pip install --upgrade pip
!pip install nvidia-pyindex
!pip install nvidia-cudnn

ERROR: You must give at least one requirement to install (see "pip help install")
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
  Using cached nvidia-cudnn-0.0.1.dev5.tar.gz (7.9 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above f

**TENSORFLOW UPGRADE**

In [27]:
# !apt-get install -y libcudnn7=7.6.5.32-1+cuda10.0 libcudnn7-dev=7.6.5.32-1+cuda10.0
# !apt-get install -y libnccl2=2.4.8-1+cuda10.0 libnccl-dev=2.4.8-1+cuda10.0

In [28]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4480393739698743960
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14410383360
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10058110403534530961
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [29]:
!nvidia-smi

Fri Nov 17 14:45:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P0    30W /  70W |    565MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [30]:
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


**TO CHECK THE GPU AVAILABILITY**

In [31]:
# Check for GPU availability
physical_devices = tf.config.list_physical_devices('GPU')
if not physical_devices:
    raise SystemError('No GPU detected')
else:
    print('GPU is available')

# Print the GPU device name and version


GPU is available


In [32]:
print(tf.test.gpu_device_name())

/device:GPU:0


In [33]:
import tensorflow as tf
if tf.test.is_gpu_available():
    print("GPU is available")


GPU is available


**TO CHECK THE MULTIPROCESSING OF LIST OF TASKS**

In [34]:
import tensorflow as tf

# Check if GPU is available
if tf.test.gpu_device_name():
    print('GPU is available')
else:
    raise SystemError('No GPU detected')

# Define a parallel task
@tf.function
def parallel_task(x):
    with tf.device('/device:GPU:0'):
        # Define your GPU-accelerated computation here
        return x * x

# Define the list of tasks
tasks = [1, 2, 3, 4, 5]

# Use the GPU-accelerated task on each element of the list
results = [parallel_task(tf.constant(task)).numpy() for task in tasks]

print(results)

GPU is available
[1, 4, 9, 16, 25]


**TO EXECUTE THE TASKS IN SEQUENTIAL AND PARALLEL WITH EXECUTION TIME**

In [35]:
import concurrent.futures
import time

# Define a task function to be executed in parallel
def parallel_task(x):
    # Replace this with your actual task logic
    result = x * x
    return result

# Number of tasks to execute in parallel
num_tasks = 5
tasks = [1, 2, 3, 4, 5]  # Example tasks, you can use different numbers

# Measure the runtime for sequential execution
start_time = time.time()
sequential_results = [parallel_task(x) for x in tasks]
sequential_runtime = time.time() - start_time

# Create a ThreadPoolExecutor for parallel execution
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Measure the runtime for parallel execution
    start_time = time.time()
    parallel_results = list(executor.map(parallel_task, tasks))
    parallel_runtime = time.time() - start_time

print("Sequential Results:", sequential_results)
print("Sequential Runtime:", sequential_runtime, "seconds")

print("Parallel Results:", parallel_results)
print("Parallel Runtime:", parallel_runtime, "seconds")

Sequential Results: [1, 4, 9, 16, 25]
Sequential Runtime: 0.00014829635620117188 seconds
Parallel Results: [1, 4, 9, 16, 25]
Parallel Runtime: 0.0006117820739746094 seconds


**TO EXECUTE THE TASKS IN SEQUENTIAL AND PARALLEL OF MATRIX MULTIPLICATION WITH EXECUTION TIME**

In [36]:
import numpy as np
import time
from concurrent.futures import ProcessPoolExecutor

def matrix_multiply(a, b):
    return np.dot(a, b)
def sequential_matrix_multiplication(a, b):
    start_time = time.time()
    result = matrix_multiply(a, b)
    end_time = time.time()
    execution_time = end_time - start_time
    return result, execution_time
def parallel_matrix_multiplication(a, b, num_workers):
    start_time = time.time()
    chunk_size = len(a) // num_workers
    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        chunks = [(a[i:i+chunk_size], b) for i in range(0, len(a), chunk_size)]
        results = list(executor.map(matrix_multiply, *zip(*chunks)))
    result = np.concatenate(results)
    end_time = time.time()
    execution_time = end_time - start_time
    return result, execution_time

# Example usage
matrix_size = 1000
matrix_a = np.random.rand(matrix_size, matrix_size)
matrix_b = np.random.rand(matrix_size, matrix_size)

# Sequential execution
sequential_result, sequential_time = sequential_matrix_multiplication(matrix_a, matrix_b)
print(f"Sequential Execution Time: {sequential_time} seconds")

# Parallel execution with 4 workers
num_workers = 4
parallel_result, parallel_time = parallel_matrix_multiplication(matrix_a, matrix_b, num_workers)
print(f"Parallel Execution Time with {num_workers} workers: {parallel_time} seconds")

# Ensure the results are the same for sequential and parallel execution
assert np.allclose(sequential_result, parallel_result), "Results do not match!"

Sequential Execution Time: 0.06389808654785156 seconds
Parallel Execution Time with 4 workers: 0.5282261371612549 seconds


**TO EXECUTE THE TASKS IN SEQUENTIAL AND PARALLEL OF MATRIX MULTIPLICATION WITH EXECUTION TIME USING GPU ACCELERATION**

In [37]:
import concurrent.futures
import tensorflow as tf
import time
import numpy as np

# Check if GPU is available
if tf.config.list_physical_devices('GPU'):
    print('GPU is available')
else:
    raise SystemError('No GPU detected')

# Define a GPU-accelerated complex task (matrix multiplication)
def gpu_complex_task(size):
    with tf.device('/device:GPU:0'):
        # Generate random matrices and perform matrix multiplication
        mat1 = tf.constant(np.random.rand(size, size), dtype=tf.float32)
        mat2 = tf.constant(np.random.rand(size, size), dtype=tf.float32)
        result = tf.matmul(mat1, mat2)
        return result.numpy()

# Number of tasks to execute in parallel
num_tasks = 5
task_sizes = [500, 750, 1000, 1250, 1500]  # Example task sizes, you can use different sizes

# Measure the runtime for sequential execution
start_time = time.time()
sequential_results = [gpu_complex_task(size) for size in task_sizes]
sequential_runtime = time.time() - start_time

# Create a ThreadPoolExecutor for parallel execution
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Measure the runtime for parallel execution
    start_time = time.time()
    parallel_results = list(executor.map(gpu_complex_task, task_sizes))
    parallel_runtime = time.time() - start_time

print("Sequential Results:", [result.shape for result in sequential_results])
print("Sequential Runtime:", sequential_runtime, "seconds")

print("Parallel Results:", [result.shape for result in parallel_results])
print("Parallel Runtime:", parallel_runtime, "seconds")

GPU is available
Sequential Results: [(500, 500), (750, 750), (1000, 1000), (1250, 1250), (1500, 1500)]
Sequential Runtime: 0.18117046356201172 seconds
Parallel Results: [(500, 500), (750, 750), (1000, 1000), (1250, 1250), (1500, 1500)]
Parallel Runtime: 0.17812228202819824 seconds


# Word Count Program

In [38]:
def sequential_word_count(text):
    word_count = 0
    in_word = False

    for char in text:
        if char.isalnum():
            in_word = True
        else:
            if in_word:
                word_count += 1
                in_word = False

    if in_word:
        word_count += 1

    return word_count

In [51]:
def parallel_word_count(text, num_processes=2):
  chunk_size = len(text) // num_processes
  chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

  with concurrent.futures.ProcessPoolExecutor(max_workers=num_processes) as executor:
    # Measure the time before running the parallel word count
    start_time = time.time()

    # Submit the count_words function for each chunk
    futures = [executor.submit(sequential_word_count, chunk) for chunk in chunks]

    # Wait for all tasks to complete
    concurrent.futures.wait(futures)

    # Get the results from completed tasks and sum them
    result = sum(future.result() for future in futures)

    # Measure the time after running the parallel word count
    end_time = time.time()

  elapsed_time = end_time - start_time

  return result, elapsed_time


In [52]:
#input_text = "This is a simple example for word count with parallel processing."
input_path = '/content/Word_Count.txt'
with open(input_path, 'r') as file:
  input_text = file.read()
#sequential_result, sequential_time = sequential_word_count(input_text)
parallel_result, parallel_time = parallel_word_count(input_text)

start_time = time.time()
result = sequential_word_count(input_text)

end_time = time.time()
elapsed_time = end_time - start_time

print(f"Sequential Word count: {result}")
print(f"Sequential Time taken: {elapsed_time:.6f} seconds")

num_threads = 16

result, elapsed_time = parallel_word_count(input_text, num_threads)
print(f"Parallel Word count: {result}")
print(f"Parallel Time taken: {elapsed_time:.6f} seconds with {num_threads} threads")

Sequential Word count: 12500
Sequential Time taken: 0.006938 seconds
Parallel Word count: 12513
Parallel Time taken: 0.436659 seconds with 16 threads
